In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print(len(text))

1115393


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("namsakaram"))
print(decode(encode("namsakaram")))

[52, 39, 51, 57, 39, 49, 39, 56, 39, 51]
namsakaram


In [6]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
n = int(0.9 * len(data)) # num of training samples
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"input {context} output {target}")

input tensor([18]) output 47
input tensor([18, 47]) output 56
input tensor([18, 47, 56]) output 57
input tensor([18, 47, 56, 57]) output 58
input tensor([18, 47, 56, 57, 58]) output 1
input tensor([18, 47, 56, 57, 58,  1]) output 15
input tensor([18, 47, 56, 57, 58,  1, 15]) output 47
input tensor([18, 47, 56, 57, 58,  1, 15, 47]) output 58


In [10]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)

print('targets')
print(yb.shape)
print(yb)

print("-----------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context} target is {target}")

inputs
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
targets
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
-----------
when input is tensor([53]) target is 59
when input is tensor([53, 59]) target is 6
when input is tensor([53, 59,  6]) target is 1
when input is tensor([53, 59,  6,  1]) target is 58
when input is tensor([53, 59,  6,  1, 58]) target is 56
when input is tensor([53, 59,  6,  1, 58, 56]) target is 47
when input is tensor([53, 59,  6,  1, 58, 56, 47]) target is 40
when input is tensor([53, 59,  6,  1, 58, 56, 47, 40]) target is 59
when input is tensor([49]) target is 43
when input is tensor([49, 43]) target is 43
when input is tensor([49, 43, 43]) target is 54
when input is tensor([49, 43, 43, 5

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import einops
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            logits = einops.rearrange(logits, 'B T C -> (B T) C')
            targets = einops.rearrange(targets, 'B T -> (B T)')
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8948, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [12]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [13]:
batch_size = 32

for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.476459264755249


In [14]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=200)[0].tolist()))


llo br. ave aviasurf my, may be ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;

Whe, oraingofoff ve!
RIfans picsheserer hee anf,
TOFonk? me ain ckntoty dedo bo


In [45]:
# version 1 - loose attn
torch.manual_seed(42)
B,T,C = 3,6,2
x = torch.randn(B,T,C)
x.shape
print(x)

tensor([[[ 1.9269,  1.4873],
         [ 0.9007, -2.1055],
         [ 0.6784, -1.2345],
         [-0.0431, -1.6047],
         [-0.7521,  1.6487],
         [-0.3925, -1.4036]],

        [[-0.7279, -0.5594],
         [-0.7688,  0.7624],
         [ 1.6423, -0.1596],
         [-0.4974,  0.4396],
         [ 0.3189, -0.4245],
         [ 0.3057, -0.7746]],

        [[ 0.0349,  0.3211],
         [ 1.5736, -0.8455],
         [-1.2742,  2.1228],
         [-1.2347, -0.4879],
         [-1.4181,  0.8963],
         [ 0.0499,  2.2667]]])


In [46]:
# x - (B, T, C)
# xbow = x

#   x
#   [[[2,1], [4,10], [42,5]]
#    [[3,3], [14,1], [4,51]]
#    [[5,11], [14,110], [4,5]]]
#
#   xbow =

In [47]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)
print(xbow)

tensor([[[ 1.9269,  1.4873],
         [ 1.4138, -0.3091],
         [ 1.1687, -0.6176],
         [ 0.8657, -0.8644],
         [ 0.5422, -0.3617],
         [ 0.3864, -0.5354]],

        [[-0.7279, -0.5594],
         [-0.7484,  0.1015],
         [ 0.0485,  0.0145],
         [-0.0880,  0.1208],
         [-0.0066,  0.0117],
         [ 0.0455, -0.1194]],

        [[ 0.0349,  0.3211],
         [ 0.8043, -0.2622],
         [ 0.1115,  0.5328],
         [-0.2251,  0.2776],
         [-0.4637,  0.4014],
         [-0.3781,  0.7122]]])


In [48]:
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print(a)
print("----")
print(b)
print("----")
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
----
tensor([[0., 9.],
        [6., 9.],
        [5., 4.]])
----
tensor([[0.0000, 9.0000],
        [3.0000, 9.0000],
        [3.6667, 7.3333]])


In [88]:
wei = torch.tril(torch.ones(T,T))
wei = wei / torch.sum(wei, 1, keepdim=True) # T x T, x is B x T x C
xbow2 = wei @ x

In [90]:
torch.allclose(xbow, xbow2)

True

In [92]:
tril = torch.tril(torch.ones(T,T))

In [94]:
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [107]:
# self attn
torch.manual_seed(1337)
B,T,C =  16,8,2
x = torch.randn(B,T,C)

# single head
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B,T,h)
print(k.shape)
q = query(x) # (B,T,h)
k_T = einops.rearrange(k, 'b t h -> b h t')
wei = torch.einsum('b t h, b h T-> b t T', q, k_T)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v

torch.Size([16, 8, 16])
